<a href="https://colab.research.google.com/github/tedjung13/KAIRI_intern_CNN/blob/main/Image_Classification_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import everything needed

In [1]:
import pandas as pd
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
import numpy as np
import shutil
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image

Import from Google Drive (need to download set into My Drive, might take a while). Also, put the train folder (not train_folder) separately from open (the zip file) and put it in My Drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Use GPU if possible

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


Make a path to the folder (that's in Google Drive already). If it's in a different directory, make sure to reflect that change after MyDrive.



In [4]:
folder_path = '/content/drive/MyDrive/train'
os.chdir(folder_path)

Load and read the CSVs: the print should be a list of all the labels.

In [6]:
# Load CSVs
train_df = pd.read_csv('/content/drive/MyDrive/train_df.csv')
label_list = train_df['label'].unique().tolist()
print(f'There are {len(label_list)} classes.')
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv')
test_list = test_df['file_name']
print(f'There are {len(test_list)} files in the testing dataset.')

There are 88 classes.
There are 2154 files in the testing dataset.


Make a new folder called train_folder where there is a folder of each label.

In [8]:
# Create folders for each label
def create_folder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
        else:
          print('Already made')
    except OSError as error:
        print(f'Error: Creating directory {directory}. Error: {error}')

for label in label_list:
    create_folder(f'/content/drive/MyDrive/train_folder/{label}')


Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made
Already made

According to the index and the label, move each image to the corresponding folder. For example, 10000.png's label is transistor-good, so the image will be found in the transistor-good folder.

In [9]:
# Move files to corresponding label folders
train_files = os.listdir('/content/drive/MyDrive/train/')

for file in train_files:
    if file.endswith(".png"):
        label = train_df.loc[train_df["file_name"] == file]["label"].values[0]
        file_source = f'/content/drive/MyDrive/train/{file}'
        file_destination = f'/content/drive/MyDrive/train_folder/{label}/'
        shutil.move(file_source, file_destination)

# Verify files are correctly moved
for label in label_list:
    label_path = f'/content/drive/MyDrive/train_folder/{label}'
    if not os.listdir(label_path):
        print(f'No files were found in directory: {label_path}')

Find the mean and standard deviations to normalize the datasets in the next step. Normalizing is done so all the data is on the same scale. For running time reasons, I resized the data to 256 x 256 instead of 1024 x 1024, but the numbers ended up being about the same. For time-saving purposes, I think you can skip this step and use the numbers I got (this code took 15+ minutes to run), but feel free to run it if you like.

In [ ]:
def get_mean_std(data_dir, batch_size = 16):
   transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()]) #Might want to change back to 1024 x 1024
   dataset = ImageFolder(data_dir, transform=transform)
   data_loader = DataLoader(dataset, batch_size = batch_size, shuffle = False)

   mean = 0.
   std = 0.
   nb_samples = 0.
   for inputs, _ in data_loader:
       batch_samples = inputs.size(0)
       inputs = inputs.view(batch_samples, inputs.size(1), -1)
       mean += inputs.mean(2).sum(0)
       std += inputs.std(2).sum(0)
       nb_samples += batch_samples
       print(f'{nb_samples} / 4277')
   mean /= nb_samples
   std /= nb_samples

   print("Mean:", mean)
   print("Standard Deviation:", std)
data_dir = '/content/drive/MyDrive/train_folder'
get_mean_std(data_dir)

16.0
32.0
48.0
64.0
80.0
96.0
112.0
128.0
144.0
160.0
176.0
192.0
208.0
224.0
240.0
256.0
272.0
288.0
304.0
320.0
336.0
352.0
368.0
384.0
400.0
416.0
432.0
448.0
464.0
480.0
496.0
512.0
528.0
544.0
560.0
576.0
592.0
608.0
624.0
640.0
656.0
672.0
688.0
704.0
720.0
736.0
752.0
768.0
784.0
800.0
816.0
832.0
848.0
864.0
880.0
896.0
912.0
928.0
944.0
960.0
976.0
992.0
1008.0
1024.0
1040.0
1056.0
1072.0
1088.0
1104.0
1120.0
1136.0
1152.0
1168.0
1184.0
1200.0
1216.0
1232.0
1248.0
1264.0
1280.0
1296.0
1312.0
1328.0
1344.0
1360.0
1376.0
1392.0
1408.0
1424.0
1440.0
1456.0
1472.0
1488.0
1504.0
1520.0
1536.0
1552.0
1568.0
1584.0
1600.0
1616.0
1632.0
1648.0
1664.0
1680.0
1696.0
1712.0
1728.0
1744.0
1760.0
1776.0
1792.0
1808.0
1824.0
1840.0
1856.0
1872.0
1888.0
1904.0
1920.0
1936.0
1952.0
1968.0
1984.0
2000.0
2016.0
2032.0
2048.0
2064.0
2080.0
2096.0
2112.0
2128.0
2144.0
2160.0
2176.0
2192.0
2208.0
2224.0
2240.0
2256.0
2272.0
2288.0
2304.0
2320.0
2336.0
2352.0
2368.0
2384.0
2400.0
2416.0
2432.0
2448

This is a custom class to create a dataset. Since the images in the training folder are defined by class, label, and state, but the images in the testing folder aren't classified in any way, I just decided to make two separate classes for the test_data and the train_data.

In [10]:
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = [f for f in os.listdir(root_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.file_list[idx])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_name  # Return image and file name

Create the datasets and transform them so some of them are flipped (for additional training). I used the values from the means and standard deviations found in the previous step to normalize.

In [12]:
# Create the training dataset
def create_datasets(batch_size):
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize([0.43303847, 0.4034577, 0.39415097], [0.18344551, 0.17549995, 0.1647388])
    ])

    # Choose the training datasets
    train_data = datasets.ImageFolder(os.path.join('/content/drive/MyDrive', 'train_folder'), train_transform)

    # Define validation set size
    valid_size = 0.95 #Ideally would be somewhere closer to 0.2, but the training data took a very long time to go through even at 0.9.

    # Get indices for training and validation sets
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # Define samplers for training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # Load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=0)

    # Load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=0)

    return train_data, train_loader, valid_loader

This is where I defined the rest of the variables for the testing data, including the test_transform, test_dataset, and test_loader.

In [15]:
train_data, train_loader, valid_loader = create_datasets(batch_size=4)

test_transform = transforms.Compose([
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize([0.43303847, 0.4034577, 0.39415097], [0.18344551, 0.17549995, 0.1647388])
    ])
test_dataset = CustomTestDataset(root_dir = '/content/drive/MyDrive/test', transform = test_transform)
print(f'There are {len(test_dataset)} images in the test dataset.')
test_loader = DataLoader(test_dataset, batch_size = 4, shuffle = False, num_workers = 0) #When actually testing, batch_size = len(test_dataset)
print(f'There are {len(test_loader)} images in the test loader.')

class_names = train_data.classes

There are 2154 images in the test dataset.
There are 539 images in the test loader.


Simple version of a Convolutional Neural Network with (Conv layer, ReLU layer, a pooling layer) x 2, then two fully conneced linear layers. This is also where a lot of the tweaks to the module itself will be made, including adding more layers or changing the transformations.

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5) #Conv layer
        self.pool = nn.MaxPool2d(2, 2) #Pooling layer
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 253 * 253, 150) #Fully connected linear layer
        self.fc2 = nn.Linear(150, len(label_list))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #Conv layer, relu, pooling layer in that order
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Call the model, loss function as Cross Entropy, optimizer as Adam with learning rate = 1e-3. We could also try to change the learning rate.

In [18]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

This is the training loop. The num_epochs can be changed as needed to see its efficiency and accuracy. The trade-off is time.

In [20]:
num_epochs = 5  # Change the number of epochs as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    samplex = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        samplex += 1
        running_loss += loss.item()
        print(f'{running_loss}, {samplex}')

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

4.440212726593018, 1.0
186.45623445510864, 2.0
297.30325746536255, 3.0
484.4775586128235, 4.0
501.40160608291626, 5.0
544.8854851722717, 6.0
591.0669350624084, 7.0
598.820972442627, 8.0
632.6691284179688, 9.0
640.7030277252197, 10.0
646.7003192901611, 11.0
650.980143070221, 12.0
654.1828672885895, 13.0
660.4033615589142, 14.0
663.0982055664062, 15.0
668.9616208076477, 16.0
671.3438699245453, 17.0
676.5627520084381, 18.0
681.8029110431671, 19.0
691.2654592990875, 20.0
695.9270842075348, 21.0
698.5564460754395, 22.0
703.5745644569397, 23.0
707.4826436042786, 24.0
711.542414188385, 25.0
713.675684928894, 26.0
716.4751303195953, 27.0
721.2222306728363, 28.0
724.7413501739502, 29.0
729.39479637146, 30.0
733.0216174125671, 31.0
736.0111453533173, 32.0
739.6233327388763, 33.0
742.9189586639404, 34.0
746.90824842453, 35.0
750.1590332984924, 36.0
753.3483834266663, 37.0
756.0788683891296, 38.0
760.7672362327576, 39.0
763.7989320755005, 40.0
767.4403605461121, 41.0
770.4639623165131, 42.0
772.87

These labels are the ones that are "good" (no anomalies)

In [21]:
correct_label = ['bottle-good', 'cable-good', 'capsule-good', 'carpet-good', 'grid-good', 'hazelnut-good', 'leather-good', 'metal_nut-good', 'pill-good', 'screw-good',
                 'tile-good', 'toothbrush-good', 'transistor-good', 'wood-good', 'zipper-good']
def is_good(label):
  return label in correct_label
good_bad_map = {i: 1 if is_good(class_name) else 0 for i, class_name in enumerate(class_names)}

This checks if the predictions equal the actual label in two parts: first the actual label, then whether it's in the correct pile of "good" or not. The number of batches can also be changed and ideally shouldn't even exist, but I found running the model with the entire training dataset took an insane amount of time, so I took a small percentage of the data.

In [22]:
model.eval()
val_running_loss = 0.0
correct = 0
total = 0
correct_good_bad = 0
total_good_bad = 0

num_batches_to_check = 4 # Ideally, this would not even be necessary.
with torch.no_grad():
    for batch_idx, (inputs, labels) in enumerate(valid_loader):
        if batch_idx >= num_batches_to_check: # Would delete this if statement if faster.
            break
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        predicted_good_bad = torch.tensor([good_bad_map[pred.item()] for pred in predicted], device=device)
        labels_good_bad = torch.tensor([good_bad_map[label.item()] for label in labels], device=device)

        print(predicted_good_bad)
        print(labels_good_bad)

        total_good_bad += labels_good_bad.size(0)
        correct_good_bad += (predicted_good_bad == labels_good_bad).sum().item()

        print(total)

    val_loss = val_running_loss/len(valid_loader)
    val_accuracy = 100 * correct / total
    val_good_bad_accuracy = 100 * correct_good_bad / total_good_bad

    print(f'{correct}, {total}')
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}%")
    print(f"Good/Bad Accuracy: {val_good_bad_accuracy}%")

tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 0, 1], device='cuda:0')
4
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 0], device='cuda:0')
8
tensor([1, 1, 1, 1], device='cuda:0')
tensor([0, 0, 0, 1], device='cuda:0')
12
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
16
11, 16
Validation Loss: 0.010999997505756814, Validation Accuracy: 68.75%
Good/Bad Accuracy: 68.75%


This is now using the testing data. it uses the model to estimate what each new image will be. The dataset was again significantly reduced for time efficiency purposes.

In [23]:
predictions = []
num_batches_to_check_test = 4 # Ideally would not be necessary
with torch.no_grad():
    for idx, (inputs, labels) in enumerate(test_loader):
        if idx >= num_batches_to_check_test: # Would delete if faster
            break
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        for i in range(len(predicted)):
            predictions.append((idx * test_loader.batch_size + i, class_names[predicted[i].item()]))
        print(f'Processed batch {idx + 1} / {len(test_loader)}')


Processed batch 1 / 539
Processed batch 2 / 539
Processed batch 3 / 539
Processed batch 4 / 539


This code will move the data into a csv called "test_predictions.csv" and download it to files from the local google.colab.

In [24]:
# Write predictions to CSV
pred_df = pd.DataFrame(predictions, columns=['Index', 'Prediction'])
pred_df.to_csv('test_predictions.csv', index=False)

# from google.colab import files
# files.download('test_predictions.csv')